In [42]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3:70b")
llm.base_url = 'http://172.28.105.30/backend'
eln_schema = llm.invoke(
    """
here is a snippet from cheat sheat to write data schemas in yaml for NOMAD (www.nomad-lab.eu):
# Description: This is a cheat sheet for the NOMAD custom schema.
#   It is a minimal example of how to define a custom schema.
#   It is not intended to be used for real data.
#   It is intended to be used as a starting point for defining a custom schema.
#   It is intended to be used as a reference for the syntax of the custom schema.

# defining the schema:
definitions:
  name: "Cheatsheet"
  sections:
    CheatSheet: # sections are in pascal case (not mandatory but best practice)
      base_sections:
      - nomad.datamodel.data.EntryData

      # adding quantities:
      quantities: # quantities are in snake case (not mandatory but best practice)

        # myquantity: test
        string:
          type: string
          description:
          m_annotations:
            eln:
              component: StringEditQuantity

        # reference: www.example.com
        url_link:
          type: string
          description:
          m_annotations:
            eln:
              component: URLEditQuantity

        # myfloatquantity: 1.0
        float:
          type: np.float64
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myfloatquantitywithunit: 1.0 fs
        float_unit:
          type: np.float64
          unit: second
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "fs"

        # myfloatquantitywithderivedunit: 1.0 eV
        float_derived_unit:
          type: np.float64
          unit: joule
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "eV"

        # myfloatquantitycomplexunit: 1.0 mA/ms^2*cm
        float_complex_unit:
          type: np.float64
          unit: ampere / second^2 * meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "milliampere / ms^2 * cm"

        # myfloatquantitywithbounds: 0.0 - 10.0 m
        float_with_bounds:
          type: np.float64
          unit: meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myintquantity: 1
        int:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myintquantitywithbounds: 0 - 10
        int_with_bounds:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myboolquantity: true
        bool:
          type: bool
          description:
          m_annotations:
            eln:
              component: BoolEditQuantity

please use this input to generate a yaml schema for the following input :
my afm experiment measures the surface roughnes of a sample. I need to also stare the position of the image in mm and the resolution

""")

yaml_file = "eln.archive.yaml"

In [44]:
import sys
from nomad.client import parse, normalize_all

# match and run the parser
archives = parse('output.schema.archive.yaml')

Schema is deprecated, use plugins. ()
parsing was not successful (parser=parsers/archive, exc_info='str' object has no attribute 'get')


AttributeError: 'str' object has no attribute 'get'

In [43]:
eln_schema

'Based on the provided cheat sheet, I\'ll help you create a YAML schema for your AFM experiment. Here\'s the generated schema:\n```\ndefinitions:\n  name: "AFM Experiment"\n  sections:\n    AFMExperiment: \n      base_sections:\n       - nomad.datamodel.data.EntryData\n\n      quantities:\n        surface_roughness:\n          type: np.float64\n          unit: nanometer\n          description: Surface roughness of the sample\n          m_annotations:\n            eln:\n              component: NumberEditQuantity\n\n        image_position_x:\n          type: np.float64\n          unit: millimeter\n          description: X-position of the image\n          m_annotations:\n            eln:\n              component: NumberEditQuantity\n\n        image_position_y:\n          type: np.float64\n          unit: millimeter\n          description: Y-position of the image\n          m_annotations:\n            eln:\n              component: NumberEditQuantity\n\n        resolution_x:\n          ty

In [7]:
eln_schema_clean = eln_schema.split('```')[1]
eln_schema_clean

'\ndefinitions:\n  name: "MySinteringSchema"\n  sections:\n    MySinteringSchema:\n      base_sections:\n       - nomad.datamodel.data.EntryData\n\n      quantities:\n        temperature:\n          type: np.float64\n          unit: degree_celsius\n          description:\n          m_annotations:\n            eln:\n              component: NumberEditQuantity\n\n        process_finished:\n          type: bool\n          description:\n          m_annotations:\n            eln:\n              component: BoolEditQuantity\n\n        user:\n          type: string\n          description:\n          m_annotations:\n            eln:\n              component: StringEditQuantity\n'

In [8]:
import yaml

# Assuming 'eln_schema' contains your YAML data as a string
yaml_file = 'output.archive.yaml'

with open(yaml_file, 'w') as file:
    data = yaml.load(eln_schema_clean, Loader=yaml.FullLoader)  # Use FullLoader to parse the YAML
    yaml.dump(data, file, default_flow_style=False)


In [49]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import yaml
import ast
import streamlit as st

llm = Ollama(model="llama3:70b")
llm.base_url = 'http://172.28.105.30/backend'

#eln_schema = llm.invoke(userprompt)
#please use this input to generate a yaml schema for the following input :

cheatsheet=    """
You are a schema generator. Below you will get a schema snippet as input. You should output only the generated schema and no other text!
here is a snippet from cheat sheet to write data schemas in yaml for NOMAD (www.nomad-lab.eu):
# Description: This is a cheat sheet for the NOMAD custom schema.
#   It is a minimal example of how to define a custom schema.
#   It is not intended to be used for real data.
#   It is intended to be used as a starting point for defining a custom schema.
#   It is intended to be used as a reference for the syntax of the custom schema.

# defining the schema:
definitions:
  name: "Cheatsheet"
  sections:
    CheatSheet: # sections are in pascal case (not mandatory but best practice)
      base_sections:
      - nomad.datamodel.data.EntryData

      # adding quantities:
      quantities: # quantities are in snake case (not mandatory but best practice)

        # myquantity: test
        string:
          type: string
          description:
          m_annotations:
            eln:
              component: StringEditQuantity

        # reference: www.example.com
        url_link:
          type: string
          description:
          m_annotations:
            eln:
              component: URLEditQuantity

        # myfloatquantity: 1.0
        float:
          type: np.float64
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myfloatquantitywithunit: 1.0 fs
        float_unit:
          type: np.float64
          unit: second
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "fs"

        # myfloatquantitywithderivedunit: 1.0 eV
        float_derived_unit:
          type: np.float64
          unit: joule
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "eV"

        # myfloatquantitycomplexunit: 1.0 mA/ms^2*cm
        float_complex_unit:
          type: np.float64
          unit: ampere / second^2 * meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "milliampere / ms^2 * cm"

        # myfloatquantitywithbounds: 0.0 - 10.0 m
        float_with_bounds:
          type: np.float64
          unit: meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myintquantity: 1
        int:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myintquantitywithbounds: 0 - 10
        int_with_bounds:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myboolquantity: true
        bool:
          type: bool
          description:
          m_annotations:
            eln:
              component: BoolEditQuantity

Please use this input to generate a yaml schema for the following input and output only the generated schema in the chat:
"""


prompt = ChatPromptTemplate.from_template(cheatsheet + "{userpromp}")
output_parser = StrOutputParser()
chain = prompt | llm | output_parser



def invokechain(userinput):
    #userinput = input("Please enter your input: ")

    return chain.invoke({"userpromp": userinput }) 
# Title of the app


In [50]:
test=invokechain("myexperiment: pressure:19mbar")

In [51]:
test

'definitions:\n  name: "MyExperiment"\n  sections:\n    MyExperiment:\n      base_sections:\n       - nomad.datamodel.data.EntryData\n      quantities:\n        pressure:\n          type: np.float64\n          unit: mbar\n          description:\n          m_annotations:\n            eln:\n              component: NumberEditQuantity'

In [55]:
with open('output2.schema.archive.yaml', 'w') as file:
    data = yaml.load(test, Loader=yaml.FullLoader)  # Use FullLoader to parse the YAML
    yaml.dump(data, file, default_flow_style=False)
#with open('output2.schema.archive.yaml', 'w') as file:
#        yaml.dump(test, file, loader=yaml.FullLoader)

In [ ]:
st.title('Invoke Chain App')

# Text input field
user_input = st.text_input("Please enter your input:")

# Button
if st.button('Invoke Chain'):
    # Call the invokechain function with the user's input
    result = invokechain(user_input)
    result = yaml.safe_load(ast.literal_eval(result))
    with open('output.schema.archive.yaml', 'w') as file:
        yaml.dump(result, file)
    # Display the result
    st.write("``` \n" + result)



In [77]:
archives[0].

EntryArchive(metadata, definitions)

In [86]:
error = None
try:
    # Replace this with the actual parse command
    parse('output.schema.archive.yaml')
except Exception as e:
    error = e
    print(f"An error occurred: {error}")

parsing was not successful (parser=parsers/archive, exc_info='degree_celsius' is not defined in the unit registry)


An error occurred: 'degree_celsius' is not defined in the unit registry


In [87]:
error

pint.errors.UndefinedUnitError('degree_celsius')

In [84]:
import traceback

try:
    parse('output.schema.archive.yaml')
except Exception:
    error_message = traceback.format_exc()
    print(error_message)

parsing was not successful (parser=parsers/archive, exc_info='bullshit' is not defined in the unit registry)


Traceback (most recent call last):
  File "/tmp/ipykernel_16478/1837658263.py", line 4, in <module>
    parse('output.schema.archive.yaml')
  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/client/processing.py", line 68, in parse
    entry_archives = parsers.run_parser(
  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/parsing/parsers.py", line 219, in run_parser
    raise e
  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/parsing/parsers.py", line 215, in run_parser
    parser.parse(mainfile_path, entry_archive, logger=logger, **kwargs)
  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/parsing/parser.py", line 561, in parse
    self.parse_file(mainfile, f, archive, logger)
  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/parsing/parser.py", line 547, in parse_file
    archive.definitions = Package.m_from_dict(
  File "/home/

In [85]:
error_message

'Traceback (most recent call last):\n  File "/tmp/ipykernel_16478/1837658263.py", line 4, in <module>\n    parse(\'output.schema.archive.yaml\')\n  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/client/processing.py", line 68, in parse\n    entry_archives = parsers.run_parser(\n  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/parsing/parsers.py", line 219, in run_parser\n    raise e\n  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/parsing/parsers.py", line 215, in run_parser\n    parser.parse(mainfile_path, entry_archive, logger=logger, **kwargs)\n  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/parsing/parser.py", line 561, in parse\n    self.parse_file(mainfile, f, archive, logger)\n  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/nomad/parsing/parser.py", line 547, in parse_file\n    archive.definitions = Package.m_from_dict(

In [69]:
import sys
from nomad.client import parse, normalize_all

# match and run the parser
archives = parse('output.schema.archive.yaml')

parsing was not successful (parser=parsers/archive, exc_info='bullshit' is not defined in the unit registry)


UndefinedUnitError: 'bullshit' is not defined in the unit registry

In [70]:
archives

[EntryArchive(metadata, definitions)]

In [68]:
parse('output.schema.archive.yaml')

parsing was not successful (parser=parsers/archive, exc_info='bullshit' is not defined in the unit registry)


UndefinedUnitError: 'bullshit' is not defined in the unit registry

In [66]:
archives.m_dict()

AttributeError: 'list' object has no attribute 'm_dict'

In [67]:
!nomad parse output.archive.yaml --show-archive

Schema is deprecated, use plugins. ()
{
  "metadata": {
    "entry_name": "MySinteringSchema",
    "entry_type": "Schema",
    "mainfile": "/home/brueckner/llm_hackathon/output.archive.yaml",
    "text_search_contents": [],
    "n_quantities": 29,
    "quantities": [
      "",
      "definitions",
      "definitions.more",
      "definitions.name",
      "definitions.section_definitions",
      "definitions.section_definitions.base_sections",
      "definitions.section_definitions.more",
      "definitions.section_definitions.name",
      "definitions.section_definitions.quantities",
      "definitions.section_definitions.quantities.description",
      "definitions.section_definitions.quantities.more",
      "definitions.section_definitions.quantities.name",
      "definitions.section_definitions.quantities.type",
      "definitions.section_definitions.quantities.unit",
      "metadata",
      "metadata.entry_name",
      "metadata.entry_type",
      "metadata.mainfile",
      "metadat

In [20]:
documents= """
here is a snippet from cheat sheat to write data schemas in yaml for NOMAD (www.nomad-lab.eu). Use this to generate data schemas from the user prompts:
# Description: This is a cheat sheet for the NOMAD custom schema.
#   It is a minimal example of how to define a custom schema.
#   It is not intended to be used for real data.
#   It is intended to be used as a starting point for defining a custom schema.
#   It is intended to be used as a reference for the syntax of the custom schema.

# defining the schema:
definitions:
  name: "Cheatsheet"
  sections:
    CheatSheet: # sections are in pascal case (not mandatory but best practice)
      base_sections:
      - nomad.datamodel.data.EntryData

      # adding quantities:
      quantities: # quantities are in snake case (not mandatory but best practice)

        # myquantity: test
        string:
          type: string
          description:
          m_annotations:
            eln:
              component: StringEditQuantity

        # reference: www.example.com
        url_link:
          type: string
          description:
          m_annotations:
            eln:
              component: URLEditQuantity

        # myfloatquantity: 1.0
        float:
          type: np.float64
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myfloatquantitywithunit: 1.0 fs
        float_unit:
          type: np.float64
          unit: second
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "fs"

        # myfloatquantitywithderivedunit: 1.0 eV
        float_derived_unit:
          type: np.float64
          unit: joule
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "eV"

        # myfloatquantitycomplexunit: 1.0 mA/ms^2*cm
        float_complex_unit:
          type: np.float64
          unit: ampere / second^2 * meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "milliampere / ms^2 * cm"

        # myfloatquantitywithbounds: 0.0 - 10.0 m
        float_with_bounds:
          type: np.float64
          unit: meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myintquantity: 1
        int:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myintquantitywithbounds: 0 - 10
        int_with_bounds:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myboolquantity: true
        bool:
          type: bool
          description:
          m_annotations:
            eln:
              component: BoolEditQuantity
"""

In [21]:
documents


'\n# here is a snippet from cheat sheat to write data schemas in yaml for NOMAD (www.nomad-lab.eu):\n# Description: This is a cheat sheet for the NOMAD custom schema.\n#   It is a minimal example of how to define a custom schema.\n#   It is not intended to be used for real data.\n#   It is intended to be used as a starting point for defining a custom schema.\n#   It is intended to be used as a reference for the syntax of the custom schema.\n\n# defining the schema:\ndefinitions:\n  name: "Cheatsheet"\n  sections:\n    CheatSheet: # sections are in pascal case (not mandatory but best practice)\n      base_sections:\n      - nomad.datamodel.data.EntryData\n\n      # adding quantities:\n      quantities: # quantities are in snake case (not mandatory but best practice)\n\n        # myquantity: test\n        string:\n          type: string\n          description:\n          m_annotations:\n            eln:\n              component: StringEditQuantity\n\n        # reference: www.example.com\

In [25]:
#import dotenv
#from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

llm = Ollama(model="llama3:70b")
llm.base_url = 'http://172.28.105.30/backend'

review_template_str = """here is a snippet from cheat sheat to write data schemas in yaml for NOMAD (www.nomad-lab.eu). Use this to generate data schemas from the user prompts:
# Description: This is a cheat sheet for the NOMAD custom schema.
#   It is a minimal example of how to define a custom schema.
#   It is not intended to be used for real data.
#   It is intended to be used as a starting point for defining a custom schema.
#   It is intended to be used as a reference for the syntax of the custom schema.

# defining the schema:
definitions:
  name: "Cheatsheet"
  sections:
    CheatSheet: # sections are in pascal case (not mandatory but best practice)
      base_sections:
      - nomad.datamodel.data.EntryData

      # adding quantities:
      quantities: # quantities are in snake case (not mandatory but best practice)

        # myquantity: test
        string:
          type: string
          description:
          m_annotations:
            eln:
              component: StringEditQuantity

        # reference: www.example.com
        url_link:
          type: string
          description:
          m_annotations:
            eln:
              component: URLEditQuantity

        # myfloatquantity: 1.0
        float:
          type: np.float64
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myfloatquantitywithunit: 1.0 fs
        float_unit:
          type: np.float64
          unit: second
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "fs"

        # myfloatquantitywithderivedunit: 1.0 eV
        float_derived_unit:
          type: np.float64
          unit: joule
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "eV"

        # myfloatquantitycomplexunit: 1.0 mA/ms^2*cm
        float_complex_unit:
          type: np.float64
          unit: ampere / second^2 * meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "milliampere / ms^2 * cm"

        # myfloatquantitywithbounds: 0.0 - 10.0 m
        float_with_bounds:
          type: np.float64
          unit: meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myintquantity: 1
        int:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myintquantitywithbounds: 0 - 10
        int_with_bounds:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myboolquantity: true
        bool:
          type: bool
          description:
          m_annotations:
            eln:
              component: BoolEditQuantity

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=review_template_str,
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="{question}",
    )
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

#chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

review_chain = review_prompt_template | llm

ModuleNotFoundError: No module named 'langchain'

In [23]:
from langchain_community.llms import Ollama
from langchain.retrieval import SimpleSearchRetriever
from langchain.chains import RetrievalAugmentedChain
llm = Ollama(model="llama3:70b")
llm.base_url = 'http://172.28.105.30/backend'
retriever = SimpleSearchRetriever(documents)
rag_chain = RetrievalAugmentedChain(lm, retriever)
#eln_schema = llm.invoke(

ModuleNotFoundError: No module named 'langchain'

In [ ]:
from langchain.llms import HuggingFaceLM
from langchain.retrieval import SimpleSearchRetriever
from langchain.chains import RetrievalAugmentedChain

# Initialize the language model
lm = HuggingFaceLM("gpt-2")

# Setup the retriever
documents = [
    {"text": "Document 1 text", "metadata": {"source": "Source A"}},
    {"text": "Document 2 text", "metadata": {"source": "Source B"}},
    # Add more documents as needed
]
retriever = SimpleSearchRetriever(documents)

# Create the RAG chain
rag_chain = RetrievalAugmentedChain(lm, retriever)

# Use the RAG chain
response = rag_chain.run("What is the explanation for RAG?")
print(response)


In [32]:
cheatsheet=    """
You are a schema generator. Below you will get a schema snippet as input. You should output only the generated schema and no other text!
here is a snippet from cheat sheet to write data schemas in yaml for NOMAD (www.nomad-lab.eu):
# Description: This is a cheat sheet for the NOMAD custom schema.
#   It is a minimal example of how to define a custom schema.
#   It is not intended to be used for real data.
#   It is intended to be used as a starting point for defining a custom schema.
#   It is intended to be used as a reference for the syntax of the custom schema.

# defining the schema:
definitions:
  name: "Cheatsheet"
  sections:
    CheatSheet: # sections are in pascal case (not mandatory but best practice)
      base_sections:
      - nomad.datamodel.data.EntryData

      # adding quantities:
      quantities: # quantities are in snake case (not mandatory but best practice)

        # myquantity: test
        string:
          type: string
          description:
          m_annotations:
            eln:
              component: StringEditQuantity

        # reference: www.example.com
        url_link:
          type: string
          description:
          m_annotations:
            eln:
              component: URLEditQuantity

        # myfloatquantity: 1.0
        float:
          type: np.float64
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myfloatquantitywithunit: 1.0 fs
        float_unit:
          type: np.float64
          unit: second
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "fs"

        # myfloatquantitywithderivedunit: 1.0 eV
        float_derived_unit:
          type: np.float64
          unit: joule
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "eV"

        # myfloatquantitycomplexunit: 1.0 mA/ms^2*cm
        float_complex_unit:
          type: np.float64
          unit: ampere / second^2 * meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              defaultDisplayUnit: "milliampere / ms^2 * cm"

        # myfloatquantitywithbounds: 0.0 - 10.0 m
        float_with_bounds:
          type: np.float64
          unit: meter
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myintquantity: 1
        int:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity

        # myintquantitywithbounds: 0 - 10
        int_with_bounds:
          type: int
          description:
          m_annotations:
            eln:
              component: NumberEditQuantity
              minValue: 0
              maxValue: 10

        # myboolquantity: true
        bool:
          type: bool
          description:
          m_annotations:
            eln:
              component: BoolEditQuantity

Please use this input to generate a yaml schema for the following input and output only the generated schema in the chat:
"""

In [33]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
llm = Ollama(model="llama3:70b")
llm.base_url = 'http://172.28.105.30/backend'

prompt = ChatPromptTemplate.from_template(cheatsheet + "{userpromp}")
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"userpromp": """ 
MySinteringSchema:
  temperature: 34.5°
  process_finished: True
  user: Sebastian"
              """})
#eln_schema = llm.invoke(userprompt)
#please use this input to generate a yaml schema for the following input :

'definitions:\n  name: "MySinteringSchema"\n  sections:\n    MySintering:\n      base_sections:\n       - nomad.datamodel.data.EntryData\n\n      quantities:\n        temperature:\n          type: np.float64\n          unit: degree Celsius\n          description:\n          m_annotations:\n            eln:\n              component: NumberEditQuantity\n\n        process_finished:\n          type: bool\n          description:\n          m_annotations:\n            eln:\n              component: BoolEditQuantity\n\n        user:\n          type: string\n          description:\n          m_annotations:\n            eln:\n              component: StringEditQuantity'

In [37]:
prompt_value = prompt.invoke({"userpromp": """
MySinteringSchema:
  temperature: 34.5°
  process_finished: True
  user: Sebastian"
              """})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='\nYou are a schema generator. Below you will get a schema snippet as input. You should output only the generated schema and no other text!\nhere is a snippet from cheat sheet to write data schemas in yaml for NOMAD (www.nomad-lab.eu):\n# Description: This is a cheat sheet for the NOMAD custom schema.\n#   It is a minimal example of how to define a custom schema.\n#   It is not intended to be used for real data.\n#   It is intended to be used as a starting point for defining a custom schema.\n#   It is intended to be used as a reference for the syntax of the custom schema.\n\n# defining the schema:\ndefinitions:\n  name: "Cheatsheet"\n  sections:\n    CheatSheet: # sections are in pascal case (not mandatory but best practice)\n      base_sections:\n      - nomad.datamodel.data.EntryData\n\n      # adding quantities:\n      quantities: # quantities are in snake case (not mandatory but best practice)\n\n        # myquantity: test\n        st

In [13]:
ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

NameError: name 'ChatPromptValue' is not defined

In [6]:
eln_schema

'Based on the provided cheat sheet, here is a YAML schema for your input data:\n\n```\ndefinitions:\n  name: "MySinteringSchema"\n  sections:\n    MySinteringData: \n      base_sections:\n       - nomad.datamodel.data.EntryData\n\n      quantities:\n        temperature:\n          type: np.float64\n          unit: degree Celsius\n          description:\n          m_annotations:\n            eln:\n              component: NumberEditQuantity\n              defaultDisplayUnit: "°"\n\n        process_finished:\n          type: bool\n          description:\n          m_annotations:\n            eln:\n              component: BoolEditQuantity\n\n        user:\n          type: string\n          description:\n          m_annotations:\n            eln:\n              component: StringEditQuantity\n```\n\nThis schema defines a custom schema for your input data, with three quantities:\n\n* `temperature`: a floating-point number with units of degrees Celsius\n* `process_finished`: a boolean value\

In [38]:
# Import the streamlit library
import streamlit as st

# Title of the app
st.title('My First Streamlit App')

# Add a button
if st.button('Say hello'):
   st.write('Hello, Streamlit!')

2024-05-07 11:47:32.785 
  command:

    streamlit run /home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [40]:
!streamlit run /home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/ipykernel_launcher.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.26.29.220:8501
  External URL: http://141.20.47.154:8501

NOTE: When using the `ipython kernel` entry point, Ctrl-C will not work.

To exit, you will have to explicitly quit this process, by either sending
"quit" from a client, or using Ctrl-\ in UNIX-like environments.

To read more about this, see https://github.com/ipython/ipython/issues/2049


To connect another client to this kernel, use:
    --existing kernel-21514.json
[IPKernelApp] ERROR | Unable to initialize signal:
Traceback (most recent call last):
  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 701, in initialize
    self.init_signal()
  File "/home/brueckner/llm_hackathon/.llmpyenv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 545, in init_signal
    signal.signal(signal.SIGINT, signal.SIG_IGN)
  File "/usr/lib/python3.9/signal.py", line 56, in signal
    handler = _sig